In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('20190301.as-rel2.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [3]:
data['provider_peer'], data['customer_peer'], data['relation_type'], data['source'] = data[0].str.split('|').str

In [4]:
data['relation_type']  = data['relation_type'].apply(lambda x: 'provider-customer' if x == '-1' else 'peer-peer')

In [5]:
links = data[['provider_peer','customer_peer','relation_type']]

In [6]:
ases = set(links['provider_peer'].values)
ases.update(links['customer_peer'].values)

In [7]:
links['link'] = list(zip(links['provider_peer'], links['customer_peer']))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
links = links[['link','relation_type']]
links_rel = {}
for index,row in links.iterrows():
    (s,t) = row['link']
    rel = row['relation_type']
    if rel == 'provider-customer':
        links_rel[tuple([s,t])] = rel
        links_rel[tuple([t,s])] = 'customer-provider'
    else: 
        if rel == 'peer-peer':
            links_rel[tuple([s,t])] = rel
            links_rel[tuple([t,s])] = 'peer-peer'
        else:
            print('Error')

In [9]:
as_graph = nx.Graph()
as_graph.add_nodes_from(ases)
for k,v in links_rel.items():
    as_graph.add_edge(k[0],k[1])

In [10]:
def shortest_path(s,t,G):
    actual_paths = []
    paths = nx.all_shortest_paths(G,s,t)
    for path in paths:
        actual_path = []
        for i in range(len(path)-1):
            l = tuple([path[i],path[i+1]])
            actual_path.append(links_rel[l])
        actual_paths.append(actual_path)
    return actual_paths

In [11]:
gate_as = ['14618','13335','37907','38895','60781','9371','16509','46606','26496','4766','45102','57127','17511','53667','19551']

In [12]:
def check_bgp_rules(path):
    up = False
    down = True
    for p in path:
        if p == 'customer-provider':
            up = True
            down = False
        if p == 'provider-customer':
            down = True
            up = False
        if down and up:
            return False
    return True

In [15]:
for s in gate_as:
    for t in gate_as:
        paths = shortest_path(s,t,as_graph)
        for path in paths:
            if not check_bgp_rules(path):
                print('Bad path')         

1
232
10
24
9
6
1
3
56
64
135
51
35
2
179
232
1
11
22
7
6
232
2
58
75
1
59
37
3
1
10
11
1
7
2
4
10
1
8
6
10
6
9
1
9
24
22
7
1
3
4
24
1
13
16
20
7
14
1
18
9
7
2
3
1
1
9
1
3
8
4
1
4
1
4
6
6
4
4
1
1
6
1
4
5
4
4
6
1
3
1
232
10
24
9
6
1
3
56
64
135
51
35
2
179
3
2
1
1
1
1
3
1
1
2
1
54
1
7
1
56
58
8
13
3
4
56
1
1
1
46
22
20
1
48
64
75
6
16
8
5
64
2
1
1
62
13
17
2
57
135
1
10
20
4
4
135
1
46
62
1
42
32
3
1
51
59
6
7
1
4
51
54
22
13
42
1
12
3
50
35
37
9
14
4
6
35
1
20
17
32
12
1
2
28
2
3
1
1
1
1
2
7
1
2
3
3
2
1
2
179
1
9
18
4
3
179
1
48
57
1
50
28
2
1
